[View in Colaboratory](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/building_resnet_model.ipynb)

In [1]:
# check the environment 

%ls

data/  sample_data/


In [3]:
'''# check if the data is in the environment 

%cd /content/data/sample/test/water
%ls'''

'# check if the data is in the environment \n\n%cd /content/data/sample/test/water\n%ls'

In [4]:
'''# check if the data is in the environment 

%cd /content/data/sample/train/road
%ls'''

'# check if the data is in the environment \n\n%cd /content/data/sample/train/road\n%ls'

# Imports 

In [5]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x57606000 @  0x7f597d6bd2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [6]:
!pip install -U bcolz

    100% |████████████████████████████████| 1.5MB 7.4MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [7]:
!pip install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 5.7MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

# GPU Settings 

In [9]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

Using gpu: True 


# Data Loader

In [0]:
# loader taken from class example: parameters to verify

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # statistics from imagenet ? 

prep1 = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
data_dir = '/content/data/sample'
#data_dir = '/content/data'

#batch_size = 4
batch_size = 64

In [0]:
# dictionary of datasets - in this case only train data set 
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'test']}

In [13]:
# train labels 
print(dsets['train'].imgs[:5]) # 5 first images and labels 
print(dsets['train'].imgs[len(dsets['train'])-5:]) # 5 last images and labels 

[('/content/data/sample/train/road/train_10001.jpg', 0), ('/content/data/sample/train/road/train_10002.jpg', 0), ('/content/data/sample/train/road/train_10038.jpg', 0), ('/content/data/sample/train/road/train_10049.jpg', 0), ('/content/data/sample/train/road/train_10071.jpg', 0)]
[('/content/data/sample/train/water/train_9968.jpg', 1), ('/content/data/sample/train/water/train_9970.jpg', 1), ('/content/data/sample/train/water/train_9991.jpg', 1), ('/content/data/sample/train/water/train_9995.jpg', 1), ('/content/data/sample/train/water/train_9999.jpg', 1)]


In [14]:
# test labels 
print(dsets['test'].imgs[:5]) # 5 first images and labels 
print(dsets['test'].imgs[len(dsets['test'])-5:]) # 5 last images and labels 

[('/content/data/sample/test/road/train_10037.jpg', 0), ('/content/data/sample/test/road/train_10099.jpg', 0), ('/content/data/sample/test/road/train_1016.jpg', 0), ('/content/data/sample/test/road/train_10216.jpg', 0), ('/content/data/sample/test/road/train_10219.jpg', 0)]
[('/content/data/sample/test/water/train_9794.jpg', 1), ('/content/data/sample/test/water/train_9837.jpg', 1), ('/content/data/sample/test/water/train_9845.jpg', 1), ('/content/data/sample/test/water/train_9847.jpg', 1), ('/content/data/sample/test/water/train_9932.jpg', 1)]


In [15]:
dset_classes = dsets['train'].classes
dset_classes # binary classes to begin 

['road', 'water']

In [0]:
# dictionary of data loaders - again only train for now 
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=True, num_workers=0)
                for x in ['train', 'test']}

In [17]:
print(len(dsets['train']))
print(len(dsets['test']))

3200
800


# Resnet

In [18]:
model_resnet = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.torch/models/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:01<00:00, 24529952.79it/s]


In [0]:
for param in model_resnet.parameters():
    param.requires_grad = False
model_resnet.fc = nn.Linear(512, 2, bias = True)

# Training Framework

In [0]:
def train_model(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()    
                
            # batch_size ?
            bs = labels.size(0)
            
            # define the loss to minimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            # define the optimizer
            optimizer = optimizer
            optimizer.zero_grad()
            # backpropagation
            loss.backward()
            optimizer.step()
            
            # predictions to get statistics 
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == labels.data)
            size += bs
        # epoch statistics     
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

# Running training epochs

# Using resnet 

In [24]:
conv_class = model_resnet
use_gpu = torch.cuda.is_available()
if use_gpu:
    conv_class = conv_class.cuda()


# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_resnet = torch.optim.Adam(model_resnet.fc.parameters(), lr = learning_rate)
optimizer_resnet = torch.optim.SGD(model_resnet.fc.parameters(), lr = learning_rate)


# and train for 5 epochs
l_t, a_t = train_model(conv_class, dset_loaders['train'], loss_fn, optimizer_resnet, n_epochs = 5)

Train - Loss: 0.0108 Acc: 0.5938
Train - Loss: 0.0094 Acc: 0.6869
Train - Loss: 0.0087 Acc: 0.7391
Train - Loss: 0.0082 Acc: 0.7722
Train - Loss: 0.0077 Acc: 0.7947


In [25]:
# and train for 10 epochs
l_t, a_t = train_model(conv_class, dset_loaders['train'], loss_fn, optimizer_resnet, n_epochs = 10)

Train - Loss: 0.0074 Acc: 0.7997
Train - Loss: 0.0072 Acc: 0.8091
Train - Loss: 0.0070 Acc: 0.8184
Train - Loss: 0.0067 Acc: 0.8197
Train - Loss: 0.0066 Acc: 0.8325
Train - Loss: 0.0065 Acc: 0.8319
Train - Loss: 0.0064 Acc: 0.8353
Train - Loss: 0.0063 Acc: 0.8359
Train - Loss: 0.0061 Acc: 0.8403
Train - Loss: 0.0061 Acc: 0.8409


# Test 

In [0]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        bs = labels.size(0)
                
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        _,preds = torch.max(outputs.data,1)
        
        # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == labels.data)
        size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [27]:
test(conv_class, dset_loaders['test'])

Test - Loss: 0.0064 Acc: 0.8425
